In [1]:
import pandas as pd
from pyprojroot import here

pd.set_option('display.max_columns', None)

## 1. Load Data

In [2]:
# Get data path 
data_path = here() /'data' /'dunnhumby - Breakfast at the Frat.xlsx'

# Get a list of all the sheet names 
xls = pd.ExcelFile(data_path)

In [9]:
# Read the data
glossary = pd.read_excel(data_path,sheet_name=xls.sheet_names[0],header=3)
dh_store_lookup = pd.read_excel(data_path,sheet_name=xls.sheet_names[1],header=1)
dh_products_lookup = pd.read_excel(data_path,sheet_name=xls.sheet_names[2],header=1)
dh_transaction_data = pd.read_excel(data_path,sheet_name=xls.sheet_names[3],header=1)

dh_transaction_data.head(4)

,WEEK_END_DATE,STORE_NUM,UPC,UNITS,VISITS,HHS,SPEND,PRICE,BASE_PRICE,FEATURE,DISPLAY,TPR_ONLY
0,2009-01-14,367,1111009477,13,13,13,18.07,1.39,1.57,0,0,1
1,2009-01-14,367,1111009497,20,18,18,27.80,1.39,1.39,0,0,0
2,2009-01-14,367,1111009507,14,14,14,19.32,1.38,1.38,0,0,0
3,2009-01-14,367,1111035398,4,3,3,14.00,3.50,4.49,0,0,1


## 2. Merge Data sets

In [10]:
# Merge transaction data with products 
data = pd.merge(dh_transaction_data,dh_products_lookup,
                left_on=['UPC'],
                right_on=['UPC'],
                how='left')

In [11]:
# Merge transaction data & products with store lookup information 
data_merge = pd.merge(data,dh_store_lookup,
                left_on=['STORE_NUM'],
                right_on=['STORE_ID'],
                how='left')

## 3. Data Analysis

3.a What is the range of prices offered on products?

In [13]:
data_merge['units per visit'] = data_merge['UNITS']/data_merge['VISITS']

In [15]:
price_products = data_merge.groupby(['DESCRIPTION'])['UNITS'].sum()

In [16]:
price_range_products = data_merge.groupby(['DESCRIPTION'])['PRICE'].aggregate(['min','max'])

In [17]:
tmp = price_range_products.merge(price_products,how='left',on='DESCRIPTION')
tmp

,min,max,UNITS
DESCRIPTION,,,
ACT KIDS BBLGUM FLUOR RNS,2.29,5.19,18819
ACT MINT A/CAV FLUOR RNS,1.75,4.99,16835
ACT RSTR CL SPLSH MINT MW,1.00,4.99,15660
COLG SPEARMINT WISP,0.50,7.99,1933
CREST PH CLN MINT RINSE,2.50,6.91,47575
CREST PH WHTG CLN MINT TP,0.81,11.46,40579
DIGIORNO THREE MEAT,4.00,7.49,144193
DIGRN PEPP PIZZA,4.17,7.49,266473
DIGRN SUPREME PIZZA,4.19,7.49,195604


3.b. Are there specific price thresholds that, if crossed, drive significant differences in sales?

3.c. What is the price elasticity of the products? What will happen to unit sales if the price changes by X%?

3.d. How are sales impacted by changing price gaps between items?

3.e. What is the impact on sales of promotions, displays, or being featured in the circular?

3.f. What is the impact on units/visit of promotions?

3.g. How do the above differ by products? By categories?

3.h. How do the above differ by geographies (e.g., southwestern states vs. midwestern states)?

3.i. How do the above differ by store price tier (e.g., upscale stores vs. value stores)?

3.j. If you were the retailer, for which products would you be more likely to lower the price to increase sales? Why?

3.k. If you were the retailer, for which products would you be more likely to increase the price to improve profits? Why?